<a href="https://colab.research.google.com/github/Ddshsp/lab/blob/main/Teeth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install pyngrok fastapi uvicorn -q
from google.colab import userdata
import os


def set_env_from_secrets(name: str) -> None:
    os.environ[name] = userdata.get(name)


set_env_from_secrets("NGROK")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 6.1 MB/s eta 0:00:00


In [3]:
%%sh
curl -sSL https://ngrok-agent.s3.amazonaws.com/ngrok.asc \
  | sudo tee /etc/apt/trusted.gpg.d/ngrok.asc >/dev/null \
  && echo "deb https://ngrok-agent.s3.amazonaws.com buster main" \
  | sudo tee /etc/apt/sources.list.d/ngrok.list \
  && sudo apt update \
  && sudo apt install ngrok

deb https://ngrok-agent.s3.amazonaws.com buster main
Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:8 https://ngrok-agent.s3.amazonaws.com buster InRelease [20.3 kB]
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,517 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2,830 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [3,614 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports/universe amd64 Packages [33.8 kB]
Ge



W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 


In [4]:
!ngrok authtoken $NGROK

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [7]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from PIL import Image
import requests
from io import BytesIO
import nest_asyncio
from transformers import pipeline
import uvicorn
import subprocess
import time

app = FastAPI()
pipe = pipeline("image-classification", model="steven123/Check_GoodBad_Teeth")
nest_asyncio.apply()

class ClassificationRequest(BaseModel):
    image_url: str
    channel_id: int

@app.post('/classify')
async def classify_image(request: ClassificationRequest):
    image_url = request.image_url
    channel_id = request.channel_id

    if not channel_id:
        raise HTTPException(status_code=400, detail="channel_id is required")

    response = requests.get(image_url)
    img = Image.open(BytesIO(response.content))

    # Classify the image
    classification_result = pipe(img)
    return {'channel_id': channel_id, 'classification_result': classification_result}

def start_ngrok():
    # Start Ngrok tunnel
    ngrok_process = subprocess.Popen(['ngrok', 'http', '8000'], stdout=subprocess.PIPE)
    time.sleep(5)  # Wait for Ngrok to start
    localhost_url = "http://localhost:4040/api/tunnels"  # Ngrok API URL
    tunnel_url = requests.get(localhost_url).json()['tunnels'][0]['public_url']
    print(f"Ngrok tunnel URL: {tunnel_url}")
    return tunnel_url, ngrok_process

if __name__ == '__main__':
    tunnel_url, ngrok_process = start_ngrok()
    print(f"FastAPI app is running at {tunnel_url}")
    uvicorn.run(app, host='0.0.0.0', port=8000)
    ngrok_process.terminate()  # Terminate Ngrok process when FastAPI app stops


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/745 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/343M [00:00<?, ?B/s]

Invalid model-index. Not loading eval results into CardData.


preprocessor_config.json:   0%|          | 0.00/228 [00:00<?, ?B/s]

Device set to use cpu


model.safetensors:   0%|          | 0.00/343M [00:00<?, ?B/s]

INFO:     Started server process [152]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


Ngrok tunnel URL: https://a62c-34-75-251-234.ngrok-free.app
FastAPI app is running at https://a62c-34-75-251-234.ngrok-free.app
INFO:     85.143.112.88:0 - "GET / HTTP/1.1" 404 Not Found
INFO:     85.143.112.88:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     34.150.150.246:0 - "POST /classify HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [152]
